In [ ]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

# list the contents of /mydrive
!ls /mydrive

#Navigate to /mydrive/Modern_Computer_Vision/
%cd /mydrive/Modern_Computer_Vision/

In [ ]:
# Import the necessary modules
import os
import csv

# Open a blank CSV file for writing
with open("file_data.csv", "w") as csvfile:
    writer = csv.writer(csvfile)

    # Write the CSV file header
    writer.writerow(["masks"])

    # Loop through the files in the images folder
    for i in range(1,5108):
        # Construct the full path to the mask file
        mask_name = ("{}_masked_image.jpg".format(i))
        #mask_path = os.path.join(masks_folder, mask_name)
        # Check if the mask file exists
              # Write the image and mask file names to the CSV file
        writer.writerow([mask_name])

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
base_directory = './High_Quality_Data/'
images_folder = os.path.join(base_directory, 'images')
masks_folder = os.path.join(base_directory, 'masks')
data = pd.read_csv(os.path.join(base_directory, 'meta_data_hq.csv'))

In [ ]:
data.describe()

In [ ]:
data.head()

#**Thresholding**

In [ ]:
for index, row in data.iterrows():
#target_size=(img_dim,img_dim
        image = load_img(os.path.join(images_folder, row['image']))
        image = img_to_array(image)
        # Create a lower and upper bound for the red color channel
        lower_green = np.array([0, 0, 0])
        upper_green = np.array([80, 255, 120])

        # Create a binary mask that only includes pixels within the red color range
        mask = cv2.inRange(image, lower_green, upper_green)

        # Mask the original image using the binary mask
        masked_image = cv2.bitwise_and(image, image, mask=mask)

        # Save the masked image
        cv2.imwrite(os.path.join(masks_folder,'masked_image_{}.jpg'.format(index)), masked_image)

#**Erosion and Dilation**

In [ ]:
# Python program to demonstrate erosion and
# dilation of images.

# Reading the input image
img = cv2.imread('masked_image_2.jpg')

# Taking a matrix of size 5 as the kernel
kernel = np.ones((3, 3), np.uint8)

# The first parameter is the original image,
# kernel is the matrix with which image is
# convolved and third parameter is the number
# of iterations, which will determine how much
# you want to erode/dilate a given image.
img_erosion = cv2.erode(img, kernel, iterations=1)
img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)

plt.figure()
fig, axes = plt.subplots(figsize=(16, 16))
plt.subplot(3 , 1, 1)
plt.imshow(img)
plt.subplot(3 , 1, 2)
plt.imshow(img_erosion)
plt.subplot(3 , 1, 3)
plt.imshow(img_dilation)

cv2.waitKey(0)

In [ ]:
data_masks = pd.read_csv(os.path.join(base_directory, 'meta_data_masks.csv'))
erosion_dilation_masks_folder = os.path.join(base_directory, 'erosion_dilation_masks')

In [ ]:
data_masks.describe()

In [ ]:
data_masks.head()

In [ ]:
for index, row in data_masks.iterrows():
#target_size=(img_dim,img_dim
        mask = load_img(os.path.join(masks_folder, row['masks']))
        mask = img_to_array(mask)

        # Taking a matrix of size 5 as the kernel
        kernel = np.ones((3, 3), np.uint8)

        # The first parameter is the original image,
        # kernel is the matrix with which image is
        # convolved and third parameter is the number
        # of iterations, which will determine how much
        # you want to erode/dilate a given image.
        mask_erosion = cv2.erode(mask, kernel, iterations=1)
        mask_dilation = cv2.dilate(mask_erosion, kernel, iterations=1)

        # Save the masked image
        cv2.imwrite(os.path.join(erosion_dilation_masks_folder,'opened_mask_{}.jpg'.format(index)), mask_dilation)

#**Black & White Forming the Thresholding Results**

In [ ]:
image = cv2.imread('masked_image_2.jpg')
blur = cv2.GaussianBlur(image, (3,3), 0)
blur_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

# create NumPy arrays from the boundaries
lower = np.array([0,0,0], dtype = "uint8")
upper = np.array([180,255,40], dtype = "uint8")

# find the colors within the specified boundaries and apply
mask = cv2.inRange(blur_hsv, lower, upper)
mask = 255 - mask
output = cv2.bitwise_and(image, image, mask = mask)

# show the images
plt.figure()
fig, axes = plt.subplots(figsize=(16, 16))
plt.subplot(2 , 1, 1)
plt.imshow(output)
plt.subplot(2 , 1, 2)
plt.imshow(mask)

cv2.waitKey()

In [ ]:
data_opened_masks = pd.read_csv(os.path.join(base_directory, 'meta_data_opened_masks.csv'))
bw_masks_folder = os.path.join(base_directory, 'bw_masks')

In [ ]:
data_opened_masks.describe()

In [ ]:
data_opened_masks.head()

In [ ]:
for index, row in data_opened_masks.iterrows():
#target_size=(img_dim,img_dim
        masks = cv2.imread(os.path.join(erosion_dilation_masks_folder, row['opened_masks']))
        opened_mask = img_to_array(masks)

        blur = cv2.GaussianBlur(opened_mask, (3,3), 0)
        blur_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        # create NumPy arrays from the boundaries
        lower = np.array([0,0,0], dtype = "uint8")
        upper = np.array([80,255,120], dtype = "uint8")

        # find the colors within the specified boundaries and apply
        mask = cv2.inRange(blur_hsv, lower, upper)
        mask = 255 - mask

        #output = cv2.bitwise_and(masks, masks, mask = mask)


        # Save the masked image
        cv2.imwrite(os.path.join(bw_masks_folder,'bw_mask_{}.jpg'.format(index)), mask)

#**General Dataset and Mask CSV File Check**

In [ ]:
data_general = pd.read_csv(os.path.join(base_directory, 'general_meta_data.csv'))

In [ ]:
data_general.describe()

In [ ]:
data_general.head()